In [1]:
import numpy as np 
import pandas as pd

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors


In [2]:
movies_df = pd.read_csv('/home/dans/Downloads/pyp/archive/movies_md.csv', low_memory=False)
keywords = pd.read_csv('/home/dans/Downloads/pyp/archive/keywords.csv', low_memory=False)
links = pd.read_csv("/home/dans/Downloads/pyp/archive/links.csv", low_memory=False)
credits = pd.read_csv("/home/dans/Downloads/pyp/archive/credits.csv")


In [3]:


mData = ['id', 'title', 'genres', 'imdb_id', 'release_date', 'vote_average']

export = movies_df[mData]

credits.drop(columns=['id'])

,cast,crew
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [4]:
exports = pd.concat([export, credits], axis=1, ignore_index=True)

exports = export.fillna('')

In [5]:
exports.to_csv('exported')

In [6]:
#exported = pd.read_csv('./exported.csv')
#exported

In [7]:

movies_data = movies_df[['id','genres','adult','runtime', 'tagline', 'vote_average','title', 'imdb_id']]

keywords_data =  keywords['keywords']

credits_data = credits[['cast', 'crew']]





In [8]:
listData = pd.concat([movies_data, keywords_data, credits_data], axis=1)

listData

,id,genres,adult,runtime,tagline,vote_average,title,imdb_id,keywords,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",False,81.0,NaN,7.7,Toy Story,tt0114709,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",False,104.0,Roll the dice and unleash the excitement!,6.9,Jumanji,tt0113497,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",False,101.0,Still Yelling. Still Fighting. Still Ready for...,6.5,Grumpier Old Men,tt0113228,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,127.0,Friends are the people who let you be yourself...,6.1,Waiting to Exhale,tt0114885,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",False,106.0,Just When His World Is Back To Normal... He's ...,5.7,Father of the Bride Part II,tt0113041,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...
46414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 10703, 'name': 'tragic love'}]",NaN,NaN
46415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,...",NaN,NaN
46416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN
46417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN


In [9]:
#movies_df['id'] = movies_df['id'].astype('string')
#movies_df['vote_average'] = movies_df['vote_average'].astype('string')
#movies_df['runtime'] = movies_df['runtime'].astype('string')

#keywords['id'] = keywords['id'].astype('string')

#moviesData = movies_df.merge(keywords, on='id')

In [10]:
columns = ['id','genres','adult','runtime', 'tagline', 'vote_average','keywords','title','cast','crew', 'imdb_id']

for column in columns:
    listData[column] = listData[column].fillna('')

In [11]:
listData['runtime'] = listData['runtime'].astype('string')
listData['vote_average'] = listData['vote_average'].astype('string')


bases = listData['genres']+""+listData['keywords']+""+listData['tagline']+""+listData['title']+''+listData['runtime']+''+listData['vote_average']

bases




0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4        [{'id': 35, 'name': 'Comedy'}][{'id': 1009, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Length: 46419, dtype: string

In [12]:
vectorizer = TfidfVectorizer()

vectorizedBases = vectorizer.fit_transform(bases)

from scipy.sparse import csr_matrix

matBases=csr_matrix(vectorizedBases)

matBases



<46419x87382 sparse matrix of type '<class 'numpy.float64'>'
	with 972644 stored elements in Compressed Sparse Row format>

In [13]:
#similarity = cosine_similarity(matBases)

nn = NearestNeighbors(metric='cosine', algorithm='brute')

nn.fit(vectorizedBases)

distances, indices = nn.kneighbors(vectorizedBases, n_neighbors=30)

distances.shape

(46419, 30)

In [14]:
userChoice = input('Enter Choice:')

userChoice


'avatar'

In [17]:
want = ['title', 'imdb_id']

moviesList = listData['title']

moviesList



0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
46414                               
46415                               
46416                               
46417                               
46418                               
Name: title, Length: 46419, dtype: object

In [22]:
userChoice = input('Enter Choice:')

getCloseMatch = difflib.get_close_matches(userChoice, moviesList)

matchedMovie = getCloseMatch[0]
    
matchedMovie

'Avatar'

In [23]:
imdbList = listData['imdb_id']

user = input('enter id')

matchID = difflib.get_close_matches(user, imdbList)[0]

matchID


'tt0083113'

In [25]:
#imdb match id
matchIDindex = listData[listData['imdb_id'] == matchID]['id'].index[0]

matchIDindex

5722

In [26]:

# Get matched Movie Id

matchedMovieIndex = listData[listData['title'] == matchedMovie]['id'].index[0]

matchedMovieIndex


14551

In [28]:
similarity_score = list(enumerate(distances[matchIDindex]))

sortedMovies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

distances[40000]


array([0.        , 0.43574862, 0.55572185, 0.59381835, 0.65204004,
       0.65882997, 0.66369007, 0.67191868, 0.67238774, 0.67570373,
       0.694564  , 0.71963849, 0.72484026, 0.73872975, 0.74682615,
       0.83473073, 0.86375092, 0.86644672, 0.8677731 , 0.86932667,
       0.87169231, 0.87247886, 0.87265575, 0.87397221, 0.87426641,
       0.87446144, 0.87537744, 0.87592133, 0.87604796, 0.8763575 ])

In [29]:
i = 1
for movie in similarity_score:
    index = movie[0]
    title = movies_df[movies_df.index == index]['title'].values[0]

    print(title)

Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
The American President
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train
Get Shorty
Copycat
Assassins
Powder
Leaving Las Vegas
Othello
Now and Then
Persuasion
The City of Lost Children
Shanghai Triad


In [30]:
def find_similar_movies(movie_title, n =29):
    movie_title = difflib.get_close_matches(movie_title, moviesList)[0]

    idx = movies_data[movies_data['title'] == movie_title].index[0]
    distances, indices = nn.kneighbors(vectorizedBases[idx], n_neighbors=n+1)
    return movies_data.iloc[indices[0][1:]][['title','vote_average']]


In [31]:
movieImdbList = listData['imdb_id']
movieImdbList[0]

'tt0114709'

In [58]:
def find_m_imdb(imdb, n = 30):
    imdb = difflib.get_close_matches(imdb, movieImdbList)[0]

    idx = movies_data[movies_data['imdb_id'] == imdb].index[0]
    distances, indices = nn.kneighbors(vectorizedBases[idx], n_neighbors=n+1)
    return listData.iloc[indices[0][1:]][['title', 'imdb_id']]

In [62]:
find_m_imdb('tt0114709')


,title,imdb_id
15348,Toy Story 3,tt0435761
25799,The Legend of Mor'du,tt2565938
1811,Small Soldiers,tt0122718
1882,Child's Play,tt0094862
2142,Toys,tt0105629
2997,Toy Story 2,tt0120363
7557,Dolls,tt0092906
21928,Toy Story of Terror!,tt2446040
31832,Toy Soldiers,tt0088292
25800,Toy Story That Time Forgot,tt3473654


In [ ]:
similar = find_similar_movies('Sphinx')

print(similar)

IndexError: positional indexers are out-of-bounds

In [ ]:
#from flask import Flask, request, jsonify

#app = Flask(__name__)

#@app.route('/recommend', methods=['GET'])
#def recommend():
 #   movie_title = request.args.get('title')
  #  n = int(request.args.get('n', 29))
   # similar_movies = find_similar_movies(movie_title, n)
   # return jsonify(similar_movies.to_dict(orient ='records'))

#if __name__ == '__main__':
 #   app.run(debug=True, port=50001)